In [1]:
%pip install pandas altair[all] si-prefix vegafusion[embed] --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import altair as alt
import pandas as pd
from si_prefix import si_format
alt.data_transformers.enable("vegafusion")
alt.renderers.enable("jupyter")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('vegafusion')

### Update bin size here

In [3]:
bin_size = 100_000

In [4]:
df_ch = pd.read_csv("data/block_sizes.csv.xz")
df_ch.columns = ['block_number', 'size']
df_ch = df_ch.sort_values('block_number').reset_index(drop=True)
df_ch.head()

,block_number,size
0,1,537
1,2,544
2,3,1079
3,4,1079
4,5,537


In [5]:
df_binned = df_ch.groupby(pd.cut(df_ch['block_number'], bins=range(0, df_ch['block_number'].max(), bin_size), precision=0), observed=False)
df_binned = df_binned['size'].sum().reset_index()
df_binned['block_number'] = pd.to_numeric(df_binned['block_number'].apply(lambda x: x.left))
df_binned.head()

,block_number,size
0,0,62335913
1,100000,69523410
2,200000,76299915
3,300000,78012843
4,400000,79124983


In [6]:
brush = alt.selection_interval(
    encodings=['x']
)
base = alt.Chart(df_binned)

text = base.mark_text(
    baseline='top',
    align='left',
    fontSize=16,
    fontWeight='bold',
    yOffset=-20
).transform_filter(
    brush
).transform_aggregate(
    total='sum(size)',
    min_range='min(block_number)',
    max_range='max(block_number)',
).transform_calculate(
    count='datum.max_range - datum.min_range',
    f_min="format(datum.min_range, ',')",
    f_max="format(datum.max_range, ',')",
    f_count="format(datum.count, ',')",
    label="'Min: ' + datum.f_min + ' | Max: ' + datum.f_max + ' | Count: ' + datum.f_count + ' | Total ' + format(datum.total, '.3s')"
).encode(
    x=alt.value(5),
    y=alt.value(0),
    text='label:N'
)

chart = base.mark_line(
    interpolate='step-after',
).encode(
    x=alt.X(
        'block_number:Q',
        axis=alt.Axis(title=f'Block number ({si_format(bin_size, precision=0, format_str="{value}{prefix}")} groups)'),
        scale=alt.Scale(domainMax=df_ch['block_number'].max())
    ),
    y=alt.Y(
        'size:Q',
        axis=alt.Axis(format='s', title='Sum of block size in Bytes')
    ),
    tooltip=['block_number']
)

In [7]:
(chart.add_params(brush) + text).properties(
    width=1000,
    height=500,
    title="Click and hold to select a block range. You can drag and zoom to change the shape of the selection."
).configure_title(
    fontSize=18,
    fontStyle='italic',
    fontWeight='lighter',
    orient='bottom',
    dy=20
)

JupyterChart()

In [8]:
import numpy.random as nr
import numpy as np

def equisum_partition(arr, p, verbose=False):
    """Adapted from https://stackoverflow.com/a/54024280"""
    ac = arr.cumsum()
    if verbose:
        print(ac)

    #sum of the entire array
    partsum = ac[-1]//p 
    if verbose:
        print(partsum)

    #generates the cumulative sums of each part
    cumpartsums = np.array(range(1,p))*partsum
    if verbose:
        print(cumpartsums)

    #finds the indices where the cumulative sums are sandwiched
    inds = np.searchsorted(ac,cumpartsums) 
    if verbose:
        print(inds)

    #split into approximately equal-sum arrays
    parts = np.split(arr,inds)
    if verbose:
        print(parts)

    return parts

def format_size(s):
    return si_format(s, precision=4, format_str="{value}{prefix}")

### Update number of jobs here

In [9]:
jobs = 25

In [10]:
numpy_p = equisum_partition(df_ch['size'].values, jobs)
for i, s in enumerate(numpy_p):
        print(f"Subset #{i}: count={len(s)}, sum={format_size(sum(s))}")

print('---\nAverage size:', format_size(sum([sum(s) for s in numpy_p])/len(numpy_p)))

ranges = []
running_count = 0
for l in numpy_p:
    ranges.append((running_count, running_count + len(l)))
    running_count += len(l)
print('Ranges:', ranges)

Subset #0: count=5729628, sum=42.7146G
Subset #1: count=2072920, sum=42.7147G
Subset #2: count=1890228, sum=42.7147G
Subset #3: count=1304775, sum=42.7147G
Subset #4: count=1023472, sum=42.7147G
Subset #5: count=826497, sum=42.7146G
Subset #6: count=564808, sum=42.7147G
Subset #7: count=589786, sum=42.7147G
Subset #8: count=478800, sum=42.7146G
Subset #9: count=446212, sum=42.7146G
Subset #10: count=492712, sum=42.7147G
Subset #11: count=549374, sum=42.7147G
Subset #12: count=540320, sum=42.7146G
Subset #13: count=436976, sum=42.7146G
Subset #14: count=381966, sum=42.7147G
Subset #15: count=341562, sum=42.7147G
Subset #16: count=288675, sum=42.7147G
Subset #17: count=260870, sum=42.7143G
Subset #18: count=258021, sum=42.7150G
Subset #19: count=246415, sum=42.7147G
Subset #20: count=231537, sum=42.7139G
Subset #21: count=272319, sum=42.7153G
Subset #22: count=286389, sum=42.7148G
Subset #23: count=451463, sum=42.7146G
Subset #24: count=577387, sum=42.7147G
---
Average size: 42.7147G
Ran

In [11]:
rules = []
flip = True
for r in ranges:
    rules.append(
        base.mark_rect(
            color='#b2d9e2' if flip else '#d3ecef',
        ).encode(
            x=alt.datum(r[0], scale=alt.Scale(domainMax=df_ch['block_number'].max())),
            x2=alt.datum(r[1]),
            size=alt.value(1),
            opacity=alt.value(0.05),
        )
    )
    flip = not flip

alt.layer(*rules, chart.interactive()).properties(
    width=1000,
    height=500
)

JupyterChart()